In [17]:
!pip list

Package                                  Version
---------------------------------------- --------------
aiohappyeyeballs                         2.4.6
aiohttp                                  3.11.12
aiosignal                                1.3.2
aniso8601                                10.0.0
annotated-types                          0.7.0
anyio                                    4.7.0
argcomplete                              3.5.3
argon2-cffi                              23.1.0
argon2-cffi-bindings                     21.2.0
arrow                                    1.3.0
asttokens                                3.0.0
async-lru                                2.0.4
attrs                                    24.2.0
autogen-agentchat                        0.4.5
autogen-core                             0.4.5
autogen-ext                              0.4.5
azure-ai-evaluation                      1.1.0
azure-ai-inference                       1.0.0b9
azure-ai-projects                        

In [ ]:
# Import necessary libraries for AutoGen agent-based chat applications
# This includes Azure AI services integration for chat completions, agent definitions,
# message handling, and UI components for building conversational AI applications
import os

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


In [ ]:
# This cell configures the Azure AI Chat Completion client
# It loads environment variables, prints a GitHub token,
# and creates a client instance to interact with the Azure OpenAI service
# Finally it sends a test question to confirm the connection works
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(override=True)

print(os.environ["GITHUB_TOKEN"])

client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of South Korea?", source="user")])
print(result)

In [ ]:
# Create an AssistantAgent for travel planning
# This agent uses the Azure AI Chat Completion client we configured earlier
# It will respond to user messages about travel planning with helpful suggestions
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

In [ ]:
# This cell defines and runs an async function to demonstrate the travel agent assistant
# It sends a message asking for vacation planning advice for Seoul and displays the response
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="Could you help me plan a sunny vacation in Seoul, South Korea?", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()

[]
source='assistant' models_usage=RequestUsage(prompt_tokens=26, completion_tokens=652) content="Sure! To tailor the perfect sunny vacation for you, I need a bit more information. However, here's a general plan that focuses on a sunny destination known for its beautiful weather, compelling activities, and stunning scenery:\n\n### Destination: Costa Rica\n\n#### Duration: 7 Days\n\n---\n\n### Day 1: Arrival in San José\n- **Flight Arrangements:** Book a flight into Juan Santamaría International Airport (SJO).\n- **Accommodation:** Stay at a mid-range hotel in San José. Consider the **Grano de Oro Hotel** for a taste of local elegance.\n- **Activity:** Explore the vibrant city of San José. Visit the Pre-Columbian Gold Museum and grab dinner at a local restaurant, such as **Restaurante Silvestre**.\n\n---\n\n### Day 2: Travel to Manuel Antonio\n- **Transportation:** Rent a car or arrange for a shuttle to Manuel Antonio (about a 3-hour drive).\n- **Accommodation:** Check into a beachfront